

---


# **VALIDADOR DE CADEIA DE CERTIFICADOS**


---


## **Funcionalidades da automação**
1. Gerar cadeia organizada: <br>
  - verifica se todos os certificados estão com o prazo de validade aceitável, caso algum esteja proximo do vencimento, será exibido um aviso no console
  - gera um arquivo txt na pasta output com a cadeia ordenada (público, intermediaria(s) e raiz), exibe no console o conteúdo do arquivo gerado.
2. Verificar validade dos certificados
  - verifica se todos os certificados estão com o prazo de validade aceitável


---


## **Instruções para configurar o programa:**
1. Clique na opção "Arquivos" no menu lateral esquerdo
2. Clique com o botão direito no diretório
3. Crie uma pasta chamada "input" e uma chamada "output" nos arquivos à esquerda da tela
4. Coloque na pasta "input" os arquivos do(s) certificado(s)
5. Aperte Ctrl + F9
6. Selecione a opção desejada no console que irá ser exibido

In [ ]:
import os
from cryptography import x509
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from datetime import datetime, timedelta

def listar_arquivos_certificados():
    """Lista todos os arquivos de certificado suportados na pasta 'input'."""
    pasta_entrada = '/content/input'
    extensoes_suportadas = ['.pem', '.cer', '.crt', '.txt']
    return [os.path.join(pasta_entrada, arquivo) for arquivo in os.listdir(pasta_entrada) if os.path.isfile(os.path.join(pasta_entrada, arquivo)) and any(arquivo.endswith(ext) for ext in extensoes_suportadas)]

def carregar_certificado(caminho_arquivo):
    """Carrega um certificado de um arquivo."""
    with open(caminho_arquivo, 'rb') as arquivo:
        try:
            return x509.load_pem_x509_certificate(arquivo.read(), default_backend())
        except ValueError:
            try:
                return x509.load_der_x509_certificate(arquivo.read(), default_backend())
            except ValueError:
                print(f"Não foi possível carregar o certificado do arquivo {caminho_arquivo}")
                return None

def verificar_validade(certificado):
    """Verifica a validade do certificado."""
    agora = datetime.utcnow()
    data_expiracao = certificado['data_expiracao']
    dias_para_expirar = (data_expiracao - agora).days
    meses_para_expirar = dias_para_expirar // 30

    if dias_para_expirar < 0:
        return "CERTIFICADO EXPIRADO", meses_para_expirar, data_expiracao
    elif dias_para_expirar <= 30:
        return "CERTIFICADO REJEITADO - 1 MÊS PARA O VENCIMENTO", meses_para_expirar, data_expiracao
    elif dias_para_expirar <= 180:
        return "ATENÇÃO CERTIFICADO PERTO DE EXPIRAR", meses_para_expirar, data_expiracao
    else:
        return "OK", meses_para_expirar, data_expiracao

def decodificar_certificado(certificado):
    """Decodifica o emissor e o assunto do certificado."""
    return {
        "assunto": certificado.subject.rfc4514_string(),
        "emissor": certificado.issuer.rfc4514_string()
    }

def ordenar_cadeia(certificados):
    """Ordena os certificados em uma cadeia na ordem: público -> intermediário -> raiz."""
    cadeia_ordenada = []
    certificado_publico = None

    for certificado in certificados:
        if certificado['assunto'] != certificado['emissor']:
            certificado_publico = certificado
            break

    if certificado_publico:
        cadeia_ordenada.append(certificado_publico)
        certificados.remove(certificado_publico)

    emissor_atual = certificado_publico['emissor']
    while certificados:
        for certificado in certificados:
            if certificado['assunto'] == emissor_atual:
                cadeia_ordenada.append(certificado)
                emissor_atual = certificado['emissor']
                certificados.remove(certificado)
                break
        else:
            break

    for certificado in certificados:
        if certificado['assunto'] == certificado['emissor']:
            cadeia_ordenada.append(certificado)

    return cadeia_ordenada

def salvar_cadeia_em_arquivo(cadeia, nome_arquivo="cadeia_certificados.txt"):
    """Salva a cadeia de certificados em um arquivo."""
    conteudo = ""
    for certificado in cadeia:
        conteudo += f"Subject {certificado['assunto']}\n"
        conteudo += f"Issuer {certificado['emissor']}\n"
        conteudo += "-----BEGIN CERTIFICATE-----\n"
        conteudo += certificado['base64']
        conteudo += "-----END CERTIFICATE-----\n"

    caminho_saida = os.path.join('/content/output', nome_arquivo)
    os.makedirs(os.path.dirname(caminho_saida), exist_ok=True)
    with open(caminho_saida, 'w') as arquivo:
        arquivo.write(conteudo)
    print(f"Cadeia de certificados salva com sucesso em: {caminho_saida}")
    print("\nConteúdo da cadeia gerada:")
    print(conteudo)

def verificar_validade_cadeia(certificados):
    """Exibe a validade de cada certificado na cadeia."""
    if len(certificados) == 1:
        certificado = certificados[0]
        status, meses, data_expiracao = verificar_validade(certificado)
        print(f"-----\nStatus: {status}\nCertificado único: {certificado['assunto']}\nMeses para expirar: {meses}\nData que vai expirar: {data_expiracao}\n-----")
        return

    cadeia_ordenada = ordenar_cadeia(certificados)
    resumo = []
    print("\nCadeia de Validade:\n")
    for indice, certificado in enumerate(cadeia_ordenada):
        status, meses, data_expiracao = verificar_validade(certificado)
        tipo_certificado = "público" if indice == 0 else "intermediário" if indice < len(cadeia_ordenada) - 1 else "raiz"
        print(f"-----\nStatus: {status}\nCertificado {tipo_certificado}: {certificado['assunto']}\nMeses para expirar: {meses}\nData que vai expirar: {data_expiracao}\n-----")
        resumo.append(status)

    mensagem_resumo = "Resumo: "
    if "CERTIFICADO EXPIRADO" in resumo:
        mensagem_resumo += "Existem certificados expirados na cadeia."
    elif "CERTIFICADO REJEITADO - 1 MÊS PARA O VENCIMENTO" in resumo:
        mensagem_resumo += "Certificados na cadeia estão muito próximos de expirar."
    elif "ATENÇÃO CERTIFICADO PERTO DE EXPIRAR" in resumo:
        mensagem_resumo += "Alguns certificados estão perto de expirar."
    else:
        mensagem_resumo += "Todos os certificados estão dentro do prazo de validade aceitável."

    print(f"\n{mensagem_resumo}\n")

def verificar_certificados():
    """Verifica todos os certificados na pasta 'input'."""
    arquivos_certificados = listar_arquivos_certificados()
    if not arquivos_certificados:
        print("Nenhum arquivo de certificado encontrado na pasta 'input'.")
        return

    certificados = []
    for arquivo in arquivos_certificados:
        certificado = carregar_certificado(arquivo)
        if certificado:
            decodificado = decodificar_certificado(certificado)
            try:
                restricoes_basicas = certificado.extensions.get_extension_for_oid(x509.oid.ExtensionOID.BASIC_CONSTRAINTS).value
                certificados.append({
                    "assunto": decodificado["assunto"],
                    "emissor": decodificado["emissor"],
                    "base64": certificado.public_bytes(serialization.Encoding.PEM).decode('utf-8').replace("-----BEGIN CERTIFICATE-----\n", "").replace("-----END CERTIFICATE-----\n", ""),
                    "ca": restricoes_basicas.ca,
                    "tamanho_caminho": restricoes_basicas.path_length,
                    "data_expiracao": certificado.not_valid_after_utc.replace(tzinfo=None)
                })
            except x509.ExtensionNotFound:
                print(f"Certificado {decodificado['assunto']} não possui a extensão 'basicConstraints'. Ignorado.")
    return certificados

def principal():
    while True:
        print("\n--------------------")
        print("Menu:")
        print("1. Gerar cadeia organizada")
        print("2. Verificar validade dos certificados")
        print("3. Sair")
        print("--------------------")
        escolha = input("Digite sua escolha: ")

        if escolha == "1":
            certificados = verificar_certificados()
            if certificados:
                if len(certificados) == 1:
                    salvar_cadeia_em_arquivo(certificados, "cadeia_certificado_unico.txt")
                else:
                    cadeia_ordenada = ordenar_cadeia(certificados)
                    salvar_cadeia_em_arquivo(cadeia_ordenada)
        elif escolha == "3":
            print("--------------------")
            print("Programa finalizado!")
            print("--------------------")
            break
        elif escolha == "2":
            certificados = verificar_certificados()
            if certificados:
                verificar_validade_cadeia(certificados)
        else:
            print("Escolha inválida. Por favor, tente novamente.")

if __name__ == "__main__":
    principal()



--------------------
Menu:
1. Gerar cadeia organizada
2. Verificar validade dos certificados
3. Sair
--------------------
Digite sua escolha: 3
--------------------
Programa finalizado!
--------------------
